# Imports

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms

C:\Users\Predator\Envs\kn_ml\lib\site-packages\torchvision\io\image.py:11: UserWarning: Failed to load image Python extension: Could not find module 'C:\Users\Predator\Envs\kn_ml\Lib\site-packages\torchvision\image.pyd' (or one of its dependencies). Try using the full path with constructor syntax.
  warn(f"Failed to load image Python extension: {e}")


# Create Fully Connected Network

In [3]:
class NN(nn.Module):
    def __init__(self, input_size, num_classes):
        super(NN,self).__init__()
        self.fc1 = nn.Linear(input_size, 50)
        self.fc2 = nn.Linear(50, num_classes)
    
    
    def forward(self,x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x
    
model = NN(784,10)
x = torch.randn(64,784)
print(model(x).shape)

torch.Size([64, 10])


# Set Device

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

# Hyperparameters

In [13]:
input_size = 784
num_classes = 10
learning_rate = 0.001
batch_size=64
num_epochs=1

# Load Data

In [7]:
train_dataset = datasets.MNIST(root='dataset/', transform = transforms.ToTensor(),download=True)
train_loader = DataLoader(dataset=train_dataset, batch_size = batch_size, shuffle=True)

test_dataset = datasets.MNIST(root='dataset/', transform = transforms.ToTensor(),download=True)
test_loader = DataLoader(dataset=test_dataset, batch_size = batch_size, shuffle=True)


  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting dataset/MNIST\raw\train-images-idx3-ubyte.gz to dataset/MNIST\raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting dataset/MNIST\raw\train-labels-idx1-ubyte.gz to dataset/MNIST\raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting dataset/MNIST\raw\t10k-images-idx3-ubyte.gz to dataset/MNIST\raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting dataset/MNIST\raw\t10k-labels-idx1-ubyte.gz to dataset/MNIST\raw



# Initialize Network

In [8]:
model = NN(input_size=input_size, num_classes = num_classes).to(device)

# Loss and Optimizer

In [10]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Train Network

In [15]:
for epoch in range(num_epochs):
    for batch_idx,(data, targets) in enumerate(train_loader):
        #Get data to cuda if possible
        data = data.to(device=device)
        targets = targets.to(device=device)
        
        #Get to correct shape
        data = data.reshape(data.shape[0],-1)
        
        #forward
        scores = model(data)
        loss = criterion(scores,targets)
        
        #backward
        optimizer.zero_grad()
        loss.backward()
        
        #gradient descent or adam step
        optimizer.step()

# Check accuracy on `Training` and `Testing` 

In [29]:
def check_accuracy(loader, model):
    num_correct=0
    num_samples=0
    model.eval()
    
    with torch.no_grad():
        for x,y in loader:
            x = x.to(device=device)
            y = y.to(device = device)
            x = x.reshape(x.shape[0],-1)
            
            scores = model(x)
            _,predictions = scores.max(1)
            num_correct +=(predictions==y).sum()
            num_samples +=predictions.size(0)
        
#         print(f'Got {num_correct} / {num_samples} with accuracy ({float(num_correct)}/{float(num_samples)})*100:.2f' )
        print(f'Got {num_correct} / {num_samples} with accuracy {(float(num_correct)/float(num_samples))*100:.2f}')
    model.train()
    return 

In [30]:
check_accuracy(train_loader, model)

Got 55804 / 60000 with accuracy 93.01


In [31]:
check_accuracy(test_loader, model)

Got 55804 / 60000 with accuracy 93.01
